<a href="https://colab.research.google.com/github/danilotpnta/UN-General-Debate-Analysis-SDGs/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UN-General-Debate-Analysis-SDGs

This project analyzes the UN General Debate Corpus from 1970 to 2023. It includes exploratory data analysis (EDA), predictive modeling, and data visualizations focusing on uncovering insights from political speeches and their connection to global challenges.

The project is divided into the following sections:

1. Data Preprocessing
2. Exploratory Data Analysis
3. Predictive Modeling
4. Data Visualization

## If running from Google Colab

In [19]:
# Clone your GitHub repository
!git clone https://github.com/danilotpnta/UN-General-Debate-Analysis-SDGs.git

# Navigate to the repository folder
%cd /content/UN-General-Debate-Analysis-SDGs

Cloning into 'UN-General-Debate-Analysis-SDGs'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 34 (delta 12), reused 22 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (34/34), 77.71 KiB | 5.55 MiB/s, done.
Resolving deltas: 100% (12/12), done.
[Errno 2] No such file or directory: '/content/UN-General-Debate-Analysis-SDGs'
/Users/datoapanta/code/UN-General-Debate-Analysis-SDGs


## Data Preprocessing

#### Donwloading the dataset

In [21]:
from importlib import reload
import utils.dataverse_downloader as dataverse_downloader

# Reload the module
reload(dataverse_downloader)

# Call the function to download all files
dataverse_downloader.download_dataset()

File already downloaded: Raw_PDFs_1946-1969.tgz
File already downloaded: Raw_PDFs_1970-1990.tgz
File already downloaded: Raw_PDFs_1991-2022.tgz
File already downloaded: README.txt
File already downloaded: Speakers_by_session.xlsx
File already downloaded: UNGDC_1946-2023.tgz
Uncompressing data...
Data already uncompressed: data/raw/Raw_PDFs_1946-1969
Data already uncompressed: data/raw/Raw_PDFs_1970-1990
Data already uncompressed: data/raw/Raw_PDFs_1991-2022
Data already uncompressed: data/raw/UNGDC_1946-2023


In [16]:
import os
import pandas as pd

# Define file paths
raw_file_path = 'data/raw/Speakers_by_session.xlsx'  # Path to the raw Excel file
processed_file_path = 'data/processed/Speakers_by_session_processed.parquet'  # Path to save processed DataFrame

# Check if the processed file exists
if os.path.exists(processed_file_path):
    # Load the already processed DataFrame using Parquet
    print(f"Loading processed data from {processed_file_path}...")
    df = pd.read_parquet(processed_file_path)
else:
    # Process the Excel file and save it to the processed folder
    print(f"Processing raw data from {raw_file_path}...")
    df = pd.read_excel(raw_file_path)
    
    # Save the processed DataFrame using Parquet in the processed folder
    print(f"Saving processed data to {processed_file_path}...")
    df.to_parquet(processed_file_path)

# Display the first 15 rows of the DataFrame
df.head(15)

Loading processed data from data/processed/Speakers_by_session_processed.parquet...


,Year,Session,ISO Code,Country,Name of Person Speaking,Post,Unnamed: 6
0,2023,78,BRA,Brazil,Luiz Inacio Lula da Silva,President,None
1,2023,78,USA,United States of America,Joseph R. Biden,President,None
2,2023,78,COL,Colombia,Gustavo Petro Urrego,President,None
3,2023,78,JOR,Jordan,Abdullah II ibn Al Hussein,King,None
4,2023,78,POL,Poland,Andrzej Duda,President,None
5,2023,78,CUB,Cuba,Miguel Diaz-Canel Bermudez,President,None
6,2023,78,TUR,Turkey,Recep Tayyip Erdogan,President,None
7,2023,78,PRT,Portugal,Marcelo Rebelo de Sousa,President,None
8,2023,78,QAT,Qatar,Tamim bin Hamad Al Thani,Amir,None
9,2023,78,ZAF,South Africa,Matamela Cyril Ramaphosa,President,None


In [23]:
import os
import zipfile
import pandas as pd

# Define file paths
base_path = 'data/raw/UNGDC_1946-2023'  # Path to the UNGDC_1946-2023 directory
processed_file_path = 'data/processed/UNGDC_1946-2023_processed.parquet'  # Path to save processed DataFrame
zip_file_path = 'data/processed/UNGDC_1946-2023_processed.zip'  # Path to save the zip of processed data

# Check if the Parquet file exists
if os.path.exists(processed_file_path):
    # Load the already processed DataFrame from the Parquet file
    print(f"Loading processed data from {processed_file_path}...")
    df = pd.read_parquet(processed_file_path)
elif os.path.exists(zip_file_path):
    # Unzip the file if the Parquet doesn't exist but the ZIP does
    print(f"Unzipping {zip_file_path}...")
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        #  Extract to the directory of the Parquet file
        zip_ref.extractall(os.path.dirname(processed_file_path))  

    # Now load the unzipped Parquet file
    print(f"Loading processed data from {processed_file_path}...")
    df = pd.read_parquet(processed_file_path)
else:
    # Data structure to hold the data
    data = []

    # Walk through the base directory and process each session
    for session_folder in os.listdir(base_path):
        session_path = os.path.join(base_path, session_folder)

        # Ensure that it is a directory
        if os.path.isdir(session_path):
            session_number = session_folder.split(' ')[1]  # Extract session number (e.g., '01')
            year = session_folder.split(' ')[-1]          # Extract the year (e.g., '1946')

            # Loop through the text files in each session directory
            for txt_file in os.listdir(session_path):
                if txt_file.endswith('.txt'):
                    file_path = os.path.join(session_path, txt_file)

                    # Read the content of the text file
                    with open(file_path, 'r', encoding='utf-8') as file:
                        content = file.read()

                    # Extract country code from the file name
                    country_code = txt_file.split('_')[0]  # Extract country code (e.g., 'CAN')

                    # Append the data to the list
                    data.append({
                        'Session': session_number,
                        'Year': year,
                        'Country Code': country_code,
                        'Content': content
                    })

    # Convert the data into a DataFrame for easy analysis
    df = pd.DataFrame(data)

    # Sort the DataFrame by Year, Session, and Country Code
    df = df.sort_values(by=['Year', 'Session', 'Country Code'], ascending=[True, True, True])

    # Reset the index for a cleaner DataFrame
    df.reset_index(drop=True, inplace=True)

    # Save the processed DataFrame as a Parquet file
    print(f"Saving processed data to {processed_file_path}...")
    df.to_parquet(processed_file_path)

    # Zip the Parquet file
    print(f"Compressing {processed_file_path} into {zip_file_path}...")
    with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(processed_file_path, os.path.basename(processed_file_path))

# Display the first few rows of the DataFrame
df.head()

Loading processed data from data/processed/UNGDC_1946-2023_processed.parquet...


,Session,Year,Country Code,Content
0,01,1946,ARG,At the resumption of the first session of the ...
1,01,1946,AUS,The General Assembly of the United Nations is ...
2,01,1946,BEL,The\tprincipal organs of the United Nations ha...
3,01,1946,BLR,As more than a year has elapsed since the Unit...
4,01,1946,BOL,Coming to this platform where so many distingu...


## Exploratory Data Analysis
